In [ ]:
import gurobipy as gp
import pandas as pd
import matplotlib.pyplot as plt
from time import time
import numpy as np

# Weighted Sum

In [ ]:
def weighted_sum_knapsack(w=1.0):
    input_data = pd.read_csv('knapsack.csv', index_col=0)
    budget = 1000

    model = gp.Model('knapsack')
    item_var = model.addVars(input_data.index, vtype=gp.GRB.BINARY, name='item')

    total_cost = gp.quicksum(input_data.loc[i, 'cost'] * item_var[i] for i in input_data.index)
    model.addConstr(total_cost <= budget, name='max_budget')

    total_profit = gp.quicksum(input_data.loc[i, 'profit'] * item_var[i] for i in input_data.index)
    total_goodwill = gp.quicksum(input_data.loc[i, 'goodwill'] * item_var[i] for i in input_data.index)

    """
    Note:
    Here, we assume the unit for profit and goodwill are the same.
    But if the two objectives are not comparable, first, we must
    normalize them to ensure they are unitless. For instance, one objective 
    is in dollars and another is number of items or total time in hours.     
    """
    model.setObjective(w * total_profit + (1 - w) * total_goodwill, gp.GRB.MAXIMIZE)
    # model.write(model.ModelName + '.lp')
    model.setParam('OutputFlag', 0)
    # model.setParam(gp.GRB.Param.MIPGap, 0)
    model.optimize()

    if model.status == gp.GRB.Status.OPTIMAL:
        profit_value = total_profit.getValue()
        goodwill_value = total_goodwill.getValue()
        print(f'Weight: {w:.3f} and (1-Weight): {(1 - w):.3f}')
        print(f'Total Profit: {profit_value} & Total Goodwill: {goodwill_value}')
        return profit_value, goodwill_value
    else:
        print('Could not find a solution!')
        return None, None

# Running Two Models and Getting the New Weights

In [ ]:
def weighted_sum_knapsack_with_new_weights(w1=1.0, w2=0.0):
    input_data = pd.read_csv('knapsack.csv', index_col=0)
    budget = 1000

    model = gp.Model('knapsack')
    item_var = model.addVars(input_data.index, vtype=gp.GRB.BINARY, name='item')

    total_cost = gp.quicksum(input_data.loc[i, 'cost'] * item_var[i] for i in input_data.index)
    model.addConstr(total_cost <= budget, name='max_budget')

    total_profit = gp.quicksum(input_data.loc[i, 'profit'] * item_var[i] for i in input_data.index)
    total_goodwill = gp.quicksum(input_data.loc[i, 'goodwill'] * item_var[i] for i in input_data.index)

    """
    Note:
    Here, we assume the unit for profit and goodwill are the same.
    But if the two objectives are not comparable, first, we must
    normalize them to ensure they are unitless. For instance, one objective 
    is in dollars and another is number of items or total time in hours.     
    """

    # Running the first model with the first set of weights
    w = w1  # resetting this to the first weight
    model.setObjective(w * total_profit + (1 - w) * total_goodwill, gp.GRB.MAXIMIZE)
    # model.write(model.ModelName + '.lp')
    model.setParam('OutputFlag', 0)
    # model.setParam(gp.GRB.Param.MIPGap, 0)
    model.optimize()

    if model.status == gp.GRB.Status.OPTIMAL:
        profit_value = total_profit.getValue()
        goodwill_value = total_goodwill.getValue()
        z_obj1_sol1 = profit_value
        z_obj2_sol1 = goodwill_value
        print("------ First Solution-------")
        print(f'Weight: {w} and (1-Weight): {(1 - w)}')
        print(f'Solution 1 Total Profit: {profit_value} & Solution 1 Total Goodwill: {goodwill_value}')
    else:
        print('Could not find a solution!')
        return None, None

    # Running the second model with the first set of weights
    w = w2  # resetting this to the second set of weights
    model.setObjective(w * total_profit + (1 - w) * total_goodwill, gp.GRB.MAXIMIZE)
    # model.write(model.ModelName + '.lp')
    model.setParam('OutputFlag', 0)
    # model.setParam(gp.GRB.Param.MIPGap, 0)
    model.optimize()

    if model.status == gp.GRB.Status.OPTIMAL:
        profit_value = total_profit.getValue()
        goodwill_value = total_goodwill.getValue()
        z_obj1_sol2 = profit_value
        z_obj2_sol2 = goodwill_value
        print("------ Second Solution-------")
        print(f'Weight: {w} and (1-Weight): {(1 - w)}')
        print(f'Solution 2 Total Profit: {profit_value} & Solution 2 Total Goodwill: {goodwill_value}')
        print("------ New Weights----------")
        new_w5 = (z_obj2_sol2 - z_obj2_sol1) / ((z_obj1_sol1 - z_obj2_sol1) - (z_obj1_sol2 - z_obj2_sol2))
        new_w6 = 1 - new_w5
        print(f'New Wgt 1: {new_w5:.3f}')
        print(f'New Wgt 2: {new_w6:.3f}')
        print("---------------")
        return profit_value, goodwill_value
    else:
        print('Could not find a solution!')
        return None, None

# Hierarchical Optimization

In [ ]:
def hierarchical_knapsack(w=1.0):
    """`w` is not used and is only added here for duck-typing."""
    input_data = pd.read_csv('knapsack.csv', index_col=0)
    budget = 1000

    diff_gap = 0.01  # amount you're willing to give up of the original objective

    model = gp.Model('knapsack')
    item_var = model.addVars(input_data.index, vtype=gp.GRB.BINARY, name='item')

    total_cost = gp.quicksum(input_data.loc[i, 'cost'] * item_var[i] for i in input_data.index)
    model.addConstr(total_cost <= budget, name='max_budget')

    # profit is the first and main objective
    total_profit = gp.quicksum(input_data.loc[i, 'profit'] * item_var[i] for i in input_data.index)
    # goodwill is the secondary objective
    total_goodwill = gp.quicksum(input_data.loc[i, 'goodwill'] * item_var[i] for i in input_data.index)

    model.setObjective(total_profit, gp.GRB.MAXIMIZE)
    # model.write(model.ModelName + '_profit.lp')
    model.setParam('OutputFlag', 0)
    model.optimize()

    profit_value = total_profit.getValue()
    print(f'Original Total Profit: {profit_value}')
    
    profit1 = total_profit.getValue()  # remember these values for summary report
    goodwill1 = total_goodwill.getValue()

    # Now, add the total profit value as a constraint and optimize for goodwill
    model.addConstr(total_profit >= profit_value * (1 - diff_gap), name='max_profit')

    # Update the objective and solve again
    model.setObjective(total_goodwill, gp.GRB.MAXIMIZE)
    # model.write(model.ModelName + '_goodwill.lp')
    model.setParam('OutputFlag', 0)
    model.optimize()

    profit2 = total_profit.getValue()  # remember these values for summary report
    goodwill2 = total_goodwill.getValue()

    if model.status == gp.GRB.Status.OPTIMAL:
        print(f'New Total Profit: {profit2} & Total Goodwill: {goodwill2}')
        profit_worse = (profit2 - profit1) / profit1
        goodwill_better = (goodwill2 - goodwill1) / goodwill1

        print("---------  Summary---------")
        print(f'Total Profit = {profit2}, and it was {profit1}. This is {profit_worse * 100:.1f}% worse. '
              f'You allowed it to be up to {diff_gap * 100:.1f}% worse.')
        print(f"Total Goodwill = {goodwill2}, and it was {goodwill1}. "
              f"This is a {goodwill_better * 100:.2f}% improvement.")
        return profit2, goodwill2
    else:
        print('Could not find a solution!')
        return None, None

In [ ]:
def plot_multi_obj_knapsack():
    profit_list, goodwill_list = [], []
    rng = np.arange(0, 1.1, 0.05)
    for w in rng:
        print(f'W = {w:.3f}')
        profit, goodwill = weighted_sum_knapsack(w)
        profit_list.append(profit)
        goodwill_list.append(goodwill)

    plt.plot(goodwill_list, profit_list)
    plt.xlabel('Total Goodwill')
    plt.ylabel('Total Profit')
    # plt.savefig('weighted_sum_knapsack.png')
    plt.show()

# Using Multi-objective Functionality in Gurobi

In [ ]:
def knapsack_gurobi_multi_objective(w=1.0):
    """`w` is not used and is only added here for duck-typing."""
    input_data = pd.read_csv('knapsack.csv', index_col=0)
    budget = 1000

    model = gp.Model('knapsack')
    item_var = model.addVars(input_data.index, vtype=gp.GRB.BINARY, name='item')

    total_cost = gp.quicksum(input_data.loc[i, 'cost'] * item_var[i] for i in input_data.index)
    model.addConstr(total_cost <= budget, name='max_budget')

    # profit is the first and main objective
    total_profit = gp.quicksum(input_data.loc[i, 'profit'] * item_var[i] for i in input_data.index)
    # goodwill is the secondary objective
    total_goodwill = gp.quicksum(input_data.loc[i, 'goodwill'] * item_var[i] for i in input_data.index)

    '''we pass the objectives, their priorities, names, relative tolerances, and weights.
    That arguments that can be passed to the `setPbjectiveN` are:
    index:    Identify which multi-objective to set
    priority: Set the priority of Nth objective (default is zero). Higher priority, higher importance
    weight:   Set the weight of Nth objective (default is 1.0). 
        if there are conflicting objectives, use 1 is for minimization and -1 for maximization. 
        If a weighted sum model needs to be solved, then weights can be passed here. 
        In the case of weighted sum, the priority of the objectives that need 
        to be summed together, should be the same.
    abstol:   Set the absolute tolerance of Nth objective (default is 1e-6)
    reltol:   Set the relative tolerance of Nth objective (default is zero)
    name:     Nth objective's name (default is no name)
    
    To learn more, check out:
    https://www.gurobi.com/documentation/9.5/refman/specifying_multiple_object.html
    https://www.gurobi.com/documentation/9.5/refman/working_with_multiple_obje.html
    '''
    model.ModelSense = gp.GRB.MAXIMIZE
    model.setObjectiveN(total_profit, index=0, priority=2, reltol=0, name='profit', weight=1)
    model.setObjectiveN(total_goodwill, index=1, priority=1, reltol=0, name='goodwill', weight=1)

    # model.write(model.ModelName + '_profit.lp')
    model.setParam('OutputFlag', 0)
    model.setParam(gp.GRB.Param.MIPGap, 0)
    model.optimize()

    if model.status == gp.GRB.Status.OPTIMAL:
        profit_value = total_profit.getValue()
        goodwill_value = total_goodwill.getValue()
        print(f'Total Profit: {profit_value} & Total Goodwill: {goodwill_value}')
        return profit_value, goodwill_value
    else:
        print('Could not find a solution!')
        return None, None

In [ ]:
if __name__ == '__main__':

  hierarchical_knapsack(w=1.0)    
#  weighted_sum_knapsack(w=1)
#   weighted_sum_knapsack_with_new_weights(w1=1, w2= 0)  #these are weights for the two objectives. So, the second weight for w1 is (1-w1)
#   plot_multi_obj_knapsack()
#   knapsack_gurobi_multi_objective()